In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import functools
from scipy.io import wavfile as wav
from scipy.fftpack import fft
import numpy as np
import os
from os.path import isfile, join

## Generate the FFT features
Locate .wav files in given directory and save the FFT feature vector in specified folder
### Current Params:
    - Split .wav file into 1 second chunks
    - 1 second chunks merged by summing 
    - Apply FFT to chunked .wav data

In [8]:
# assume we are at music-generation/fft/
cwd = os.getcwd()
path_dir_wav_files = os.path.join(cwd, 'wav')
wav_filepaths = getWavPathsInDir(path_dir_wav_files)
recordWavFFTFeatures(wav_filepaths, os.path.join(cwd, 'features'))

/Users/valerian/anaconda2/lib/python2.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


## Load FFT features
Attempt to load the features for each .wav file by searching in the specified folder

In [ ]:
# TODO
def LoadFFTFeatures(wav_filepaths, features_folder):
    """
    args
    ----
    wav_filepaths: [String] - a list of .wav files 
    features_folder: String - directory path to folder where .npy features saved for .wav files
    
    returns
    ----
    { String : np.array } - Dictionnary where keys are the .wav filepath and values are the features loaded from file
    """
    
    # use np.load(filename + '.npy')

In [4]:
def apply_fft_merge(filepath, seconds=1.0, merge_fn=lambda x,y: x + y):
    """
    args
    ----
    filepath: String - filepath to wav file to sample by merged intervals
    seconds: float - the ratio of seconds to group the sound segments into
    
    returns
    ----
    fft_out: np.array - fft transform on the merged wav data
    """
    rate, data = wav.read(filepath)
    
    samples, channels = np.shape(data)
    cluster_size = int(math.ceil(rate * seconds))
    total_clusters = int(samples // cluster_size)
    
    merged_array = np.zeros((total_clusters, channels))

    for i in range(total_clusters):
        start_index = i*cluster_size
        end_index = (i*cluster_size) + cluster_size
        
        cluster_value = functools.reduce(merge_fn, data[start_index:end_index, :])
        merged_array[i,:] = cluster_value
    
    fft_out = fft(merged_array)
    return fft_out

In [5]:
def getWavPathsInDir(wd=os.getcwd()):
    """
    args
    ----
    wd: String - (default: current working dir) the directory to look for .wav files
    
    returns
    ---
    wavs_filepaths: [String] - an array of filepaths of .wav files in a directory
    """
    return [os.path.join(wd, filename) for filename in os.listdir(wd) if filename.endswith('.wav')]

In [6]:
def recordWavFFTFeatures(wav_filepaths, save_dir):
    """
    args
    ----
    wav_filepaths: [String] - filepaths to wav file to perform FFT and save as features
    save_dir: os.path - the path to a dir to save the features to
    """

    for fp in wav_filepaths:
        filename = os.path.basename(fp).replace('.wav', '') #trim the extension
        fft_features = apply_fft_merge(fp)
        np.save(os.path.join(save_dir, filename), fft_features)